In [1]:
import pandas as pd
import spacy
import random
from spacy.training.example import Example
from spacy.util import minibatch
from spacy.pipeline.tok2vec import DEFAULT_TOK2VEC_MODEL


In [2]:
# Lee el archivo CSV en un DataFrame de pandas
df = pd.read_csv("dataset_entrenamiento.csv",thousands=',')
# Obtén 1000 filas aleatorias del DataFrame
random_rows = df.sample(n=1000, random_state=42)
# Encuentra las clases únicas en la columna "clase"
unique_classes = random_rows["target"].unique()
unique_classes

# Crea la lista de tuplas con las etiquetas de clases
data_list = []
for _, row in random_rows.iterrows():
    title = row["title"]
    clase = row["target"]
    cats = {cls: 1 if cls == clase else 0 for cls in unique_classes}
    data_list.append((title, {"cats": cats}))
data_list

[('“Escampe: ruta de espacios artísticos independientes” invita a programación de actividades liberadas en siete comunas',
  {'cats': {'cultura': 1,
    'deportes': 0,
    'medioambiente': 0,
    'salud': 0,
    'cienciatecnologia': 0,
    'internacional': 0,
    'economia': 0,
    'politica': 0,
    'entretenimiento': 0,
    'sociedad': 0}}),
 ('Aula Records publica disco homenaje que incluye obra inédita de Guillermo Rifo',
  {'cats': {'cultura': 1,
    'deportes': 0,
    'medioambiente': 0,
    'salud': 0,
    'cienciatecnologia': 0,
    'internacional': 0,
    'economia': 0,
    'politica': 0,
    'entretenimiento': 0,
    'sociedad': 0}}),
 ('¿Listo para el FIP 2023?, presentará 80 espectáculos',
  {'cats': {'cultura': 1,
    'deportes': 0,
    'medioambiente': 0,
    'salud': 0,
    'cienciatecnologia': 0,
    'internacional': 0,
    'economia': 0,
    'politica': 0,
    'entretenimiento': 0,
    'sociedad': 0}}),
 ('El Torneo 2023 del fútbol chileno tiene su primer DT despedido:

In [3]:
# Datos de entrenamiento
TRAIN_DATA = data_list
# Inicializar spaCy con el modelo de lenguaje es_core_news_sm
nlp = spacy.load("es_core_news_sm")
train_examples = []

for example in TRAIN_DATA:
    train_examples.append(Example.from_dict(nlp.make_doc(example[0]), example[1]))

def get_examples():
    return train_examples
model = {
            "@architectures": "spacy.TextCatCNN.v2",
            "exclusive_classes": True,
            "tok2vec": DEFAULT_TOK2VEC_MODEL,
        }
# Crear un componente TextCategorizer con un modelo CNN
textcat = nlp.add_pipe("textcat", config={"model": model})
textcat.initialize(get_examples)
from spacy.util import compounding

# Entrenar el modelo
with nlp.select_pipes(enable="textcat"):
    optimizer = nlp.begin_training()
    for epoch in range(100):
        losses = {}
        random.shuffle(TRAIN_DATA)
        # Dividir los datos en lotes y actualizar el modelo
        for batch in minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
            texts, annotations = zip(*batch)
            example = []
            # Actualizar el modelo con iteraciones
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            nlp.update(example, drop=0.5, losses=losses)
        print(losses)

{'textcat': 22.01863044500351}
{'textcat': 21.34402632713318}
{'textcat': 18.92950757034123}
{'textcat': 15.61745246546343}
{'textcat': 12.740335505921394}
{'textcat': 10.368622548179701}
{'textcat': 8.16743857815527}
{'textcat': 6.735198464664791}
{'textcat': 4.96210461107313}
{'textcat': 3.553510334904729}
{'textcat': 2.7387579855386264}
{'textcat': 2.270908568816189}
{'textcat': 2.1615925147230337}
{'textcat': 1.9573492909176347}
{'textcat': 1.5021513484488656}
{'textcat': 0.9975769668449129}
{'textcat': 0.980934673157905}
{'textcat': 0.9621471046154983}
{'textcat': 1.1749346412993709}
{'textcat': 0.7043126846268765}
{'textcat': 0.8882393884206357}
{'textcat': 0.607951735147144}
{'textcat': 0.8805914577962577}
{'textcat': 0.5495316678775783}
{'textcat': 0.7171639603438243}
{'textcat': 0.745941049277078}
{'textcat': 0.8115170691762048}
{'textcat': 0.7007405506262949}
{'textcat': 0.7449676090970856}
{'textcat': 0.5927141207978704}
{'textcat': 0.36507077934253446}
{'textcat': 0.6887198

In [4]:
# Guardar el modelo entrenado en disco
nlp.to_disk("model_1000_CNNv2")

In [5]:
# Cargar el modelo entrenado
nlp_loaded = spacy.load("model_1000_CNNv2")#modelo_clasificador_noticias")


In [ ]:
# Realizar predicciones en nuevos titulares
nuevos_titulares = [
    "El partido de fútbol se jugará mañana",
    "Nueva legislación de impuestos aprobada por el gobierno",
    "Nueva aplicación móvil para seguimiento de ejercicios",
    "Grave accidente de tráfico en la autopista deja varios heridos",
    "Avión se estrella en zona montañosa, sobreviven todos los pasajeros",
    "Incendio en fábrica causa evacuación masiva en la ciudad",
    "Naufragio de embarcación turística en el mar, varios desaparecidos",
    "Accidente de tren en la estación central, servicios suspendidos",
    "Nuevas regulaciones para reducir las emisiones de gases contaminantes en la industria",
    "Los científicos advierten sobre el aumento del nivel del mar debido al deshielo de los polos",
    "Proyecto de reforestación masiva busca combatir la deforestación en la Amazonia",
    "La contaminación del aire en las ciudades alcanza niveles peligrosos para la salud",
    "Descubren una nueva especie en peligro de extinción en la selva tropical",
    "Acuerdo internacional para proteger los océanos y conservar la vida marina",
    "Innovadora tecnología de reciclaje convierte plásticos en materiales reutilizables",
    "Cambio climático: las temperaturas extremas se vuelven más frecuentes y devastadoras",
    "Los esfuerzos de conservación salvan a una especie en peligro crítico de desaparecer",
    "La energía solar y eólica supera a los combustibles fósiles como fuente de electricidad",
]

for titular in nuevos_titulares:
    doc = nlp_loaded(titular)
    print(f"Titular: {titular}")
    print("Categorías predichas:")
    for label, score in doc.cats.items():
        print(f"{label}: {score:.4f}")

In [9]:


noticias = 'test_data2.csv'  
df = pd.read_csv(noticias, encoding='utf-8')

resultados = []
for index in range(len(df)):
    text = df.iloc[index]['text']
    if text:
        doc = nlp_loaded(text)
       
        max_label = max(doc.cats, key=doc.cats.get)
        max_score = doc.cats[max_label]
        resultados.append({
                'text': text,  
                'category': max_label,})
resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv("output_desafio1.csv", index=False, encoding='utf-8-sig')

PermissionError: [Errno 13] Permission denied: 'output_desafio1.csv'